# Import Neccesary packages

In [1]:
#All neccesary classes for project

#general
import seaborn as sns
import scipy.stats as ss
import pandas as pd
import numpy as np
import os
import pickle
import time
import tensorflow as tf
import shutil
from datetime import datetime
import os
import random
import sys
import threading


#for utility packages
from Utilities.utilities import import_data
from Utilities.utilities import DataFrameSelector
from Utilities.utilities import CategoricalEncoder

from functools import partial

#image manipulation
from PIL import Image as PI
from PIL import ImageFilter
from resizeimage import resizeimage
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#Import Custom Functions
from Utilities.model_builder import get_image
from Utilities.model_builder import get_file_lists
from Utilities.model_builder import parse_record
from Utilities.model_builder import get_batch
from Utilities.model_builder import build_iterator
from Utilities.model_builder import get_values
from Utilities.models import log_dir_build
from Utilities.utilities import generate_image
from Utilities.utilities import generate_image_array
from Utilities.blur_tool import blur_images_in_directory
from Utilities.resize_tool import resize_images_in_directory
from Utilities.blur_tool import blur_and_print_image
from Utilities.utilities import import_data
from Utilities.utilities import DataFrameSelector
from Utilities.utilities import CategoricalEncoder
from Utilities.utilities import display_scores
from Utilities.bounded_file_label_extractor import get_files_and_labels
from Utilities.bounded_box_record_maker import process_bounded_image_files

#for preprocessing
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Import the data and split into training and validation data

Here we are going to import the raw data and create a training and validation set

In [2]:
raw_data_csv = "image_table_data.csv"
data_folder = "D:/Machine_Learning/Datasets/corpus_learning"
image_folder = "D:/Machine_Learning/Datasets/corpus_learning/raw_data/"
tf_record_folder = "D:/Machine_Learning/Datasets/corpus_learning/tf_records/"
 
train_folder = "D:/Machine_Learning/Datasets/corpus_learning/training_data/"
train_csv = "D:/Machine_Learning/Datasets/corpus_learning/" + "train_data.csv"
val_folder = "D:/Machine_Learning/Datasets/corpus_learning/validation_data/"
val_csv = "D:/Machine_Learning/Datasets/corpus_learning/" + "val_data.csv"

In [9]:
raw_data = import_data(raw_data_csv, data_folder)

In [204]:
#Split data into train and test sets
train, test = train_test_split(raw_data, test_size=0.25, random_state=42)
train = train.reset_index().drop(['index'], axis=1)
test = test.reset_index().drop(['index'], axis=1)

#print the CSV into desired folders
train.to_csv(train_csv)
test.to_csv(val_csv)

In [205]:
#move all corresponding images to correct test and validation folder

#move training data
for x in train['Filename']:
    shutil.move(image_folder + x, train_folder + x)
    
#move validation data
for x in test['Filename']:
    shutil.move(image_folder + x, val_folder + x)


# Make Tensorflow Records

Here we are going to make tfrecords using the data we just divided up

In [3]:
train = import_data(train_csv,'')
val = import_data(val_csv, '')

In [4]:
train_files, train_labels = get_files_and_labels(train, train_folder,'Filename')
val_files, val_labels = get_files_and_labels(val, val_folder,'Filename')

In [7]:
#Make train_records
process_bounded_image_files("train", train_files, train_labels, 2, 2, tf_record_folder)

In [8]:
#Make test_records
process_bounded_image_files("validation", val_files, val_labels, 2, 2, tf_record_folder)


Launching 2 threads for spacings: [[0, 160], [160, 320]]
2018-11-14 17:01:48.442719 [thread 1]: Wrote 160 images to D:/Machine_Learning/Datasets/corpus_learning/tf_records/validation-00001-of-00002
2018-11-14 17:01:48.444720 [thread 1]: Wrote 160 images to 160 shards.
2018-11-14 17:01:48.463720 [thread 0]: Wrote 160 images to D:/Machine_Learning/Datasets/corpus_learning/tf_records/validation-00000-of-00002
2018-11-14 17:01:48.465719 [thread 0]: Wrote 160 images to 160 shards.
2018-11-14 17:01:49.276720: Finished writing all 320 images in data set.


# Dev Area

Use hts to test different ammounts of blur etc